<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/socio-economic/HousingValuesGeo_NYC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pygeohash

  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6162 sha256=7bd1e5b23561b782bccc4d496d0d0cd6d0d1ef9b74ee400c6140662499ee6e9c
  Stored in directory: /root/.cache/pip/wheels/3f/5f/14/989d83a271207dda28232746d63e737a2dbd88ea7f7a9db807
Successfully built pygeohash


In [3]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pygeohash as pgh
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/'

In [6]:
NYC_prop = pd.read_csv(DATA_ROOT + 'pluto_20v6.csv', usecols=['assesstot', 'latitude', 'longitude'])

In [8]:
NYC_prop.head()

,assesstot,latitude,longitude
0,900.0,40.840277,-73.780735
1,805050.0,40.644117,-73.886114
2,27780.0,40.899491,-73.848780
3,962550.0,40.714403,-73.962452
4,6541196.0,40.717140,-73.964803


In [9]:
NYC_prop["GEOHASH"] = [pgh.encode(row["latitude"],row["longitude"]) for idx, row in NYC_prop.iterrows()]

In [10]:
NYC_prop.head()

,assesstot,latitude,longitude,GEOHASH
0,900.0,40.840277,-73.780735,dr7834dxb788
1,805050.0,40.644117,-73.886114,dr5rnxjc0u88
2,27780.0,40.899491,-73.848780,dr72xt8by9wx
3,962550.0,40.714403,-73.962452,dr5rsgy4tfy6
4,6541196.0,40.717140,-73.964803,dr5rsuk6x1jd


In [27]:
sum(NYC_prop.GEOHASH.isna())

0

In [11]:
nyc_crime = pd.read_csv(DATA_ROOT + 'covid_analysis/nyc_crime.csv')
nyc_crime.shape

(306656, 36)

In [13]:
nyc_crime.tail()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
306651,453871844,90,BROOKLYN,01/01/2020,10:00:00,01/01/2020,10:05:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,FRONT OF,ASSAULT 3 & RELATED OFFENSES,NaN,PATROL BORO BKLYN NORTH,101.0,ASSAULT 3,STREET,01/01/2020,NaN,UNKNOWN,UNKNOWN,U,NaN,25-44,UNKNOWN,M,1000178,195552,40.703416,-73.942555,"(40.70341571400007, -73.94255491999998)",POINT (-73.94255491999998 40.70341571400007)
306652,931476633,101,QUEENS,01/01/2020,02:30:00,01/01/2020,02:35:00,COMPLETED,BEACH 41ST STREET-BEACH CHANNEL DRIVE,46468.0,2.0,N.Y. HOUSING POLICE,344,MISDEMEANOR,INSIDE,ASSAULT 3 & RELATED OFFENSES,NaN,PATROL BORO QUEENS SOUTH,101.0,ASSAULT 3,RESIDENCE - PUBLIC HOUSING,01/02/2020,NaN,25-44,BLACK,F,NaN,25-44,BLACK,F,1047410,156429,40.595821,-73.772573,"(40.595820856000046, -73.77257302099997)",POINT (-73.77257302099997 40.595820856000046)
306653,889424078,115,QUEENS,01/01/2020,06:10:00,01/01/2020,06:20:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,FRONT OF,ASSAULT 3 & RELATED OFFENSES,NaN,PATROL BORO QUEENS NORTH,101.0,ASSAULT 3,STREET,01/01/2020,NaN,UNKNOWN,UNKNOWN,U,NaN,18-24,WHITE HISPANIC,F,1017168,214730,40.756008,-73.881184,"(40.75600770400007, -73.88118438299993)",POINT (-73.88118438299993 40.75600770400007)
306654,951280302,42,BRONX,01/01/2020,00:30:00,01/01/2020,00:36:00,COMPLETED,WEBSTER,865.0,2.0,N.Y. HOUSING POLICE,106,FELONY,INSIDE,FELONY ASSAULT,NaN,PATROL BORO BRONX,109.0,"ASSAULT 2,1,UNCLASSIFIED",RESIDENCE - PUBLIC HOUSING,01/01/2020,NaN,45-64,BLACK,F,NaN,25-44,BLACK,M,1009394,242828,40.833155,-73.909139,"(40.833154599000075, -73.90913931099993)",POINT (-73.90913931099993 40.833154599000075)
306655,187190867,113,QUEENS,01/01/2020,07:01:00,01/01/2020,07:11:00,COMPLETED,NaN,NaN,3.0,PORT AUTHORITY,236,MISDEMEANOR,NaN,DANGEROUS WEAPONS,NaN,PATROL BORO QUEENS SOUTH,782.0,"WEAPONS, POSSESSION, ETC",AIRPORT TERMINAL,01/01/2020,NaN,25-44,BLACK HISPANIC,M,NaN,UNKNOWN,UNKNOWN,E,1046315,187088,40.679981,-73.776234,"(40.67998073800004, -73.77623390699993)",POINT (-73.77623390699993 40.67998073800004)


In [15]:
nyc_crime["GEOHASH"] = [pgh.encode(row["Latitude"], row["Longitude"]) for idx, row in nyc_crime.iterrows()]

In [23]:
nyc_crime.tail()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,GEOHASH,LOCAL_PROP_VALUES
306651,453871844,90,BROOKLYN,01/01/2020,10:00:00,01/01/2020,10:05:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,FRONT OF,ASSAULT 3 & RELATED OFFENSES,NaN,PATROL BORO BKLYN NORTH,101.0,ASSAULT 3,STREET,01/01/2020,NaN,UNKNOWN,UNKNOWN,U,NaN,25-44,UNKNOWN,M,1000178,195552,40.703416,-73.942555,"(40.70341571400007, -73.94255491999998)",POINT (-73.94255491999998 40.70341571400007),dr5rt3udtddu,NaN
306652,931476633,101,QUEENS,01/01/2020,02:30:00,01/01/2020,02:35:00,COMPLETED,BEACH 41ST STREET-BEACH CHANNEL DRIVE,46468.0,2.0,N.Y. HOUSING POLICE,344,MISDEMEANOR,INSIDE,ASSAULT 3 & RELATED OFFENSES,NaN,PATROL BORO QUEENS SOUTH,101.0,ASSAULT 3,RESIDENCE - PUBLIC HOUSING,01/02/2020,NaN,25-44,BLACK,F,NaN,25-44,BLACK,F,1047410,156429,40.595821,-73.772573,"(40.595820856000046, -73.77257302099997)",POINT (-73.77257302099997 40.595820856000046),dr5wcq0ry3w8,143857.197802
306653,889424078,115,QUEENS,01/01/2020,06:10:00,01/01/2020,06:20:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,FRONT OF,ASSAULT 3 & RELATED OFFENSES,NaN,PATROL BORO QUEENS NORTH,101.0,ASSAULT 3,STREET,01/01/2020,NaN,UNKNOWN,UNKNOWN,U,NaN,18-24,WHITE HISPANIC,F,1017168,214730,40.756008,-73.881184,"(40.75600770400007, -73.88118438299993)",POINT (-73.88118438299993 40.75600770400007),dr5ryg3kgwkp,NaN
306654,951280302,42,BRONX,01/01/2020,00:30:00,01/01/2020,00:36:00,COMPLETED,WEBSTER,865.0,2.0,N.Y. HOUSING POLICE,106,FELONY,INSIDE,FELONY ASSAULT,NaN,PATROL BORO BRONX,109.0,"ASSAULT 2,1,UNCLASSIFIED",RESIDENCE - PUBLIC HOUSING,01/01/2020,NaN,45-64,BLACK,F,NaN,25-44,BLACK,M,1009394,242828,40.833155,-73.909139,"(40.833154599000075, -73.90913931099993)",POINT (-73.90913931099993 40.833154599000075),dr72q1mn25xq,NaN
306655,187190867,113,QUEENS,01/01/2020,07:01:00,01/01/2020,07:11:00,COMPLETED,NaN,NaN,3.0,PORT AUTHORITY,236,MISDEMEANOR,NaN,DANGEROUS WEAPONS,NaN,PATROL BORO QUEENS SOUTH,782.0,"WEAPONS, POSSESSION, ETC",AIRPORT TERMINAL,01/01/2020,NaN,25-44,BLACK HISPANIC,M,NaN,UNKNOWN,UNKNOWN,E,1046315,187088,40.679981,-73.776234,"(40.67998073800004, -73.77623390699993)",POINT (-73.77623390699993 40.67998073800004),dr5x3jtf14hc,106410.812500


In [30]:
# define function to apply across the GEOHASH of 
from collections import defaultdict

# setting up home value dict (to 600 meter precision) with geohash. 100 meters was giving too many 0 values (almost 1% of rows)
home_value_dict = defaultdict(list)
for idx, row in NYC_prop.iterrows():
    if pd.notna(row['assesstot']):
      home_value_dict[row['GEOHASH'][:6]].append(row['assesstot'])


In [31]:
home_value_dict['dr5rt3']

[19500.0,
 111840.0,
 247050.0,
 878400.0,
 297900.0,
 182700.0,
 414000.0,
 517050.0,
 66840.0,
 249750.0,
 92999.0,
 94501.0,
 34200.0,
 94920.0,
 349200.0,
 149850.0,
 3421350.0,
 59340.0,
 6300.0,
 450450.0,
 334350.0,
 516600.0,
 86460.0,
 499950.0,
 970650.0,
 237150.0,
 335250.0,
 66300.0,
 790650.0,
 478350.0,
 700650.0,
 74820.0,
 986855.0,
 1591650.0,
 125550.0,
 45960.0,
 66300.0,
 415800.0,
 204750.0,
 597150.0,
 438300.0,
 1174500.0,
 86400.0,
 212400.0,
 70320.0,
 70320.0,
 71520.0,
 371250.0,
 88740.0,
 91440.0,
 125550.0,
 65640.0,
 86460.0,
 73260.0,
 108450.0,
 113400.0,
 91800.0,
 66840.0,
 66720.0,
 1021050.0,
 41700.0,
 167400.0,
 147300.0,
 145800.0,
 581850.0,
 793350.0,
 211950.0,
 66300.0,
 68460.0,
 414900.0,
 543150.0,
 104400.0,
 371250.0,
 494550.0,
 1836900.0,
 162900.0,
 75840.0,
 66300.0,
 278100.0,
 94950.0,
 189900.0,
 711000.0,
 518400.0,
 1567350.0,
 267750.0,
 719100.0,
 451800.0,
 80550.0,
 97650.0,
 50400.0,
 12840.0,
 60660.0,
 13680.0,
 16380.0,

In [32]:
# function for getting values from the dictionary from geohash
from statistics import mean

def get_prop_values(geohash, home_values):
  try:
    mean_property_value = mean(home_values[geohash[:6]])
  except:
    mean_property_value = 0

  return mean_property_value

In [33]:
# prop_values_test = get_prop_values(origin, home_value_dict)
# this should create list of the property values near 
LOCAL_PROP_VALUES = [get_prop_values(row["GEOHASH"], home_value_dict) for idx, row in nyc_crime.iterrows()]

In [34]:
nyc_crime['LOCAL_PROP_VALUES'] = LOCAL_PROP_VALUES

In [37]:
nyc_crime.head(10)

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,GEOHASH,LOCAL_PROP_VALUES
0,972326799,81,NaN,09/28/2020,21:27:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/28/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,1003904,186483,40.678516,-73.929143,"(40.67851591200008, -73.92914304899993)",POINT (-73.92914304899993 40.67851591200008),dr5rmtq3w0gr,2.616263e+05
1,376304873,52,NaN,09/27/2020,19:13:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/27/2020,NaN,45-64,WHITE HISPANIC,M,NaN,UNKNOWN,BLACK HISPANIC,M,1017928,258050,40.874906,-73.878224,"(40.87490600500007, -73.87822380899996)",POINT (-73.87822380899996 40.87490600500007),dr72wc598dex,4.867931e+05
2,299326203,75,NaN,09/21/2020,01:21:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/21/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,1021234,181211,40.663990,-73.866692,"(40.66399002800006, -73.86669235099998)",POINT (-73.86669235099998 40.66399002800006),dr5rr4euvm4j,2.951804e+05
3,674946147,121,NaN,09/15/2020,08:46:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/15/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,938430,170972,40.635845,-74.165090,"(40.63584491100005, -74.165090337)",POINT (-74.165090337 40.63584491100005),dr5r0md8uns2,4.717530e+04
4,416422620,101,NaN,09/08/2020,13:50:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/08/2020,NaN,<18,BLACK,M,NaN,18-24,BLACK,M,1049837,157548,40.598875,-73.763823,"(40.59887464700005, -73.76382298499993)",POINT (-73.76382298499993 40.59887464700005),dr5wcqycd6g0,1.438572e+05
5,119130081,68,NaN,09/07/2020,06:50:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/07/2020,NaN,45-64,WHITE HISPANIC,M,NaN,45-64,WHITE,M,978972,168177,40.628290,-74.019016,"(40.62829017100006, -74.01901584699993)",POINT (-74.01901584699993 40.62829017100006),dr5r5shuzyjr,1.663550e+05
6,873514553,41,NaN,09/04/2020,21:30:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/04/2020,NaN,25-44,WHITE HISPANIC,M,NaN,25-44,WHITE HISPANIC,M,1015461,237096,40.817403,-73.887242,"(40.81740254300007, -73.88724185399997)",POINT (-73.88724185399997 40.81740254300007),dr72nwsfw7r6,4.480467e+05
7,383718152,46,NaN,09/03/2020,02:40:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/03/2020,NaN,18-24,BLACK,M,NaN,25-44,BLACK,M,1008497,248717,40.849321,-73.912359,"(40.84932069000007, -73.91235948099995)",POINT (-73.91235948099995 40.84932069000007),dr72qh6st1q1,5.888114e+05
8,990133707,70,NaN,09/02/2020,20:40:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/02/2020,NaN,NaN,NaN,NaN,NaN,18-24,BLACK,M,996193,176571,40.651323,-73.956961,"(40.65132343000005, -73.95696100099997)",POINT (-73.95696100099997 40.65132343000005),dr5rm065jguv,6.223408e+05
9,458329800,73,NaN,09/02/2020,00:51:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,NaN,09/02/2020,NaN,NaN,NaN,NaN,NaN,18-24,BLACK,M,1006621,185426,40.675608,-73.919351,"(40.67560823700006, -73.91935098699997)",POINT (-73.91935098699997 40.67560823700006),dr5rmuv8d6by,2.769874e+05

In [42]:
sum(nyc_crime.LOCAL_PROP_VALUES == 0)

224

Only 224 of over 300K rows is this 0. A spot check of one of them shows a very deserted place on Riker's Island, so not surprising!

In [40]:
nyc_crime[nyc_crime.LOCAL_PROP_VALUES == 0].head(10) # 

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,GEOHASH,LOCAL_PROP_VALUES
1027,390204599,41,BRONX,03/08/2020,14:14:00,03/08/2020,14:14:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,359,MISDEMEANOR,INSIDE,OFFENSES AGAINST PUBLIC ADMINI,NaN,PATROL BORO BRONX,759.0,"PUBLIC ADMINISTATION,UNCLASS M",OTHER,09/22/2020,NaN,18-24,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1016268,227533,40.791152,-73.884372,"(40.791151867000046, -73.884371919)",POINT (-73.884371919 40.791151867000046),dr72n9z19mg6,0.0
1931,813376489,63,BROOKLYN,09/23/2020,19:30:00,09/23/2020,19:45:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,348,MISDEMEANOR,NaN,VEHICLE AND TRAFFIC LAWS,NaN,PATROL BORO BKLYN SOUTH,916.0,LEAVING SCENE-ACCIDENT-PERSONA,STREET,09/23/2020,NaN,UNKNOWN,UNKNOWN,U,NaN,25-44,BLACK,M,1013773,151962,40.583736,-73.893713,"(40.583735714000056, -73.89371346699994)",POINT (-73.89371346699994 40.583735714000056),dr5qys01rstt,0.0
3397,736433661,44,BRONX,09/25/2020,10:10:00,09/25/2020,13:00:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,578,VIOLATION,INSIDE,HARRASSMENT 2,NaN,PATROL BORO BRONX,637.0,"HARASSMENT,SUBD 1,CIVILIAN",PUBLIC BUILDING,09/25/2020,NaN,18-24,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1016268,227533,40.791152,-73.884372,"(40.791151867000046, -73.884371919)",POINT (-73.884371919 40.791151867000046),dr72n9z19mg6,0.0
4837,333559103,5,MANHATTAN,08/24/2020,13:02:00,08/24/2020,13:02:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,359,MISDEMEANOR,INSIDE,OFFENSES AGAINST PUBLIC ADMINI,NaN,PATROL BORO MAN SOUTH,759.0,"PUBLIC ADMINISTATION,UNCLASS M",OTHER,09/29/2020,NaN,25-44,BLACK,M,NaN,UNKNOWN,UNKNOWN,M,1016268,227533,40.791152,-73.884372,"(40.791151867000046, -73.884371919)",POINT (-73.884371919 40.791151867000046),dr72n9z19mg6,0.0
7576,266894109,41,BRONX,03/25/2020,19:01:00,03/25/2020,19:01:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,359,MISDEMEANOR,INSIDE,OFFENSES AGAINST PUBLIC ADMINI,NaN,PATROL BORO BRONX,759.0,"PUBLIC ADMINISTATION,UNCLASS M",OTHER,09/22/2020,NaN,18-24,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1016268,227533,40.791152,-73.884372,"(40.791151867000046, -73.884371919)",POINT (-73.884371919 40.791151867000046),dr72n9z19mg6,0.0
10085,759498101,44,BRONX,09/30/2020,18:49:00,NaN,NaN,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,107,FELONY,INSIDE,BURGLARY,NaN,PATROL BORO BRONX,223.0,"BURGLARY,RESIDENCE,NIGHT",RESIDENCE - APT. HOUSE,09/30/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK,M,1016268,227533,40.791152,-73.884372,"(40.791151867000046, -73.884371919)",POINT (-73.884371919 40.791151867000046),dr72n9z19mg6,0.0
12953,905393148,71,BROOKLYN,09/17/2020,19:00:00,09/18/2020,08:00:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,341,MISDEMEANOR,NaN,PETIT LARCENY,NaN,PATROL BORO BKLYN SOUTH,357.0,"LARCENY,PETIT OF VEHICLE ACCES",STREET,09/18/2020,NaN,UNKNOWN,UNKNOWN,U,NaN,UNKNOWN,UNKNOWN,D,1016265,227533,40.791152,-73.884383,"(40.79115187800004, -73.88438275299995)",POINT (-73.88438275299995 40.79115187800004),dr72n9z19jg5,0.0
17463,833582425,41,BRONX,04/01/2020,15:35:00,04/01/2020,15:35:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,359,MISDEMEANOR,INSIDE,OFFENSES AGAINST PUBLIC ADMINI,NaN,PATROL BORO BRONX,759.0,"PUBLIC ADMINISTATION,UNCLASS M",OTHER,09/15/2020,NaN,18-24,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1016268,227533,40.791152,-73.884372,"(40.791151867000046, -73.884371919)",POINT (-73.884371919 40.791151867000046),dr72n9z19mg6,0.0
18467,815694671,5,MANHATTAN,09/05/2020,10:12:00,09/05/2020,10:22:00,COMPLETED,NaN,NaN,72.0,DEPT OF CORRECTIONS,106,FELONY,INSIDE,FELONY ASSAULT,NaN,PATROL BORO MAN SOUTH,106.0,ASSAULT POLICE/PEACE OFFICER,OTHER,09/15/2020,NaN,25-44,WHITE HISPANIC,M,NaN,UNKNOWN,UNKNOWN,M,1016268,227533,40.7911

In [43]:
nyc_crime['LOCAL_PROP_VALUES'].describe()

count    3.066560e+05
mean     2.337846e+06
std      1.684878e+07
min      0.000000e+00
25%      1.754780e+05
50%      4.371159e+05
75%      1.365302e+06
max      7.240287e+09
Name: LOCAL_PROP_VALUES, dtype: float64

In [45]:
nyc_crime.to_csv(DATA_ROOT + 'data_prelim/nyc_crime_local_prop_values.csv', index=False)